# Product Recommendation with Feathr

This notebook illustrates the use of Feathr Feature Store to create a model that predict users' rating for different products for a e-commerce website.

### Model Problem Statement
The e-commerce website has collected past user ratings for various products. The website also collected data about user and product, like user age, product category etc. Now we want to predict users' product rating for new product so that we can recommend the new product to users that give a high rating for those products.

### Feature Creation Illustration
In this example, our observation data has compound entity key where a record is uniquely identified by `user_id` and `product_id`. With that, we can think about three types of features:
1. **User features** that are different for different users but are the same for different products. For example, user age is different for different users but it's product-agnostic.
2. **Product features** that are different for different products but are the same for all the users.
3. **User-to-product** features that are different for different users AND different products. For example, a feature to represent if the user has bought this product before or not.

In this example, we will focus on the first two types of features. After we train a model based on those features, we predict the product ratings that users will give for the products.

The feature creation flow is as below:
![Feature Flow](https://github.com/feathr-ai/feathr/blob/main/docs/images/product_recommendation_advanced.jpg?raw=true)

## 2. Config Feathr Client

In [1]:
import glob
import os
import tempfile
from datetime import datetime, timedelta
from math import sqrt

import pandas as pd
from pyspark.sql import DataFrame
# from interpret.provider import InlineProvider
# from interpret import set_visualize_provider

# set_visualize_provider(InlineProvider())

import feathr
from feathr import (
    FeathrClient,
    BOOLEAN, FLOAT, INT32, ValueType,
    Feature, DerivedFeature, FeatureAnchor,
    BackfillTime, MaterializationSettings,
    FeatureQuery, ObservationSettings,
    RedisSink,
    INPUT_CONTEXT, HdfsSource,
    WindowAggTransformation,
    TypedKey,
)
from feathr.datasets.constants import (
    PRODUCT_RECOMMENDATION_USER_OBSERVATION_URL,
    PRODUCT_RECOMMENDATION_USER_PROFILE_URL,
    PRODUCT_RECOMMENDATION_USER_PURCHASE_HISTORY_URL,
    PRODUCT_RECOMMENDATION_PRODUCT_DETAIL_URL,
)
from feathr.datasets.utils import maybe_download
from feathr.utils.config import generate_config
from feathr.utils.job_utils import get_result_df

print(f"Feathr version: {feathr.__version__}")

Feathr version: 1.0.0


In [2]:
os.environ['SPARK_LOCAL_IP'] = "127.0.0.1"
os.environ['REDIS_PASSWORD'] = "password"
PROJECT_NAME = "product_recommendation"

In [3]:
from pathlib import Path


import glob
jar_name = glob.glob("./*.jar")[0]
print(f"Found jar file at {jar_name}")

yaml_config = f"""
api_version: 1
project_config:
  project_name: {PROJECT_NAME}

spark_config:
  # choice for spark runtime. Currently support: azure_synapse, databricks, local
  spark_cluster: 'local'
  spark_result_output_parts: '1'
  local:
    master: 'local[*]'
    spark.sql.shuffle.partitions: '12'
    feathr_runtime_location: "{jar_name}"

online_store:
  mongodb:
    # mongodb configs
    host: '127.0.0.1'
    port: 27017
    database: _feathr
    
  redis:
    # Redis configs to access Redis cluster
    host: '172.20.0.3'
    port: 6379
    ssl_enabled: False
    cluster_mode: True


feature_registry:
  # The API endpoint of the registry service
  api_endpoint: "http://127.0.0.1:8081/api/v1"
"""
feathr_workspace_folder = Path(f"./{PROJECT_NAME}_feathr_config.yaml")
feathr_workspace_folder.parent.mkdir(exist_ok=True, parents=True)
feathr_workspace_folder.write_text(yaml_config)

Found jar file at ./vnpt_feathr-0.0.1.jar


714

### Initialize Feathr Client

In [4]:
from pathlib import Path
feathr_workspace_folder = Path(f"./{PROJECT_NAME}_feathr_config.yaml")
client = FeathrClient(str(feathr_workspace_folder))

2024-11-11 17:30:09.075 | INFO     | feathr.utils._env_config_reader:get:62 - Config secrets__azure_key_vault__name is not found in the environment variable, configuration file, or the remote key value store. Returning the default value: None.
2024-11-11 17:30:09.087 | INFO     | feathr.client:_construct_mongodb_client:621 - fffffffffffffffffffffffffffffffffffffffffffffff
2024-11-11 17:30:09.087 | INFO     | feathr.client:_construct_mongodb_client:622 - True
2024-11-11 17:30:09.088 | INFO     | feathr.client:_construct_mongodb_client:624 - 127.0.0.1
2024-11-11 17:30:09.088 | INFO     | feathr.client:_construct_mongodb_client:625 - 27017
2024-11-11 17:30:09.090 | INFO     | feathr.client:_construct_mongodb_client:626 - _feathr
2024-11-11 17:30:09.091 | INFO     | feathr.client:_construct_mongodb_client:627 - True
2024-11-11 17:30:09.091 | INFO     | feathr.client:_construct_mongodb_client:628 - 9999999999999999999999999999999999999999999999999999999999999999999999
2024-11-11 17:30:09.09

yyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyy
2222222222222222222222222222222222222222222222222222222222222222222222222222222222


## 3. Prepare Datasets

In [5]:
# Download datasets
WORKING_DIR = PROJECT_NAME
user_observation_file_path = f"{WORKING_DIR}/user_observation.csv"
user_profile_file_path = f"{WORKING_DIR}/user_profile.csv"
user_purchase_history_file_path = f"{WORKING_DIR}/user_purchase_history.csv"
product_detail_file_path = f"{WORKING_DIR}/product_detail.csv"
# maybe_download(
#     src_url=PRODUCT_RECOMMENDATION_USER_OBSERVATION_URL,
#     dst_filepath=user_observation_file_path,
# )
# maybe_download(
#     src_url=PRODUCT_RECOMMENDATION_USER_PROFILE_URL,
#     dst_filepath=user_profile_file_path,
# )
# maybe_download(
#     src_url=PRODUCT_RECOMMENDATION_USER_PURCHASE_HISTORY_URL,
#     dst_filepath=user_purchase_history_file_path,
# )
# maybe_download(
#     src_url=PRODUCT_RECOMMENDATION_PRODUCT_DETAIL_URL,
#     dst_filepath=product_detail_file_path,
# )

# In local mode, we can use the same data path as the source.
user_observation_source_path = user_observation_file_path
user_profile_source_path = user_profile_file_path
user_purchase_history_source_path = user_purchase_history_file_path
product_detail_source_path = product_detail_file_path

In [6]:
!mkdir {PROJECT_NAME}

mkdir: cannot create directory ‘product_recommendation’: File exists


In [7]:
import pandas as pd
import numpy as np
import random

In [8]:
user_ids = np.arange(1, 500 + 1)
product_ids = np.arange(1, 100 + 1)

In [9]:
# Create a DataFrame with random data for each column
data = {
    'user_id': np.random.permutation(user_ids),
    'age': np.random.randint(18, 65, size=len(user_ids)),
    'gift_card_balance': np.random.randint(0, 2000, size=len(user_ids)),
    'tax_rate': np.random.randint(1, 6, size=len(user_ids)),
    'number_of_credit_cards': np.random.randint(1, 3, size=len(user_ids))
}

df = pd.DataFrame(data)

# Hiển thị DataFrame
df.to_csv(user_profile_file_path, index=False)

In [10]:
# Create random data for each column
data = {
    'product_id': np.random.permutation(product_ids) ,
    'price': np.random.uniform(1, 200, size=len(product_ids)),
    'quantity': np.random.randint(1, 100, size=len(product_ids))
}

# Create a DataFrame
df = pd.DataFrame(data)

# Save to CSV
df.to_csv(product_detail_source_path, index=False)

In [11]:
from datetime import datetime, timedelta

# Function to generate random dates
def random_date(start, end):
    return (start + timedelta(
        seconds=random.randint(0, int((end - start).total_seconds())))).strftime('%d/%m/%Y')

# Set the number of samples you want to generate


In [12]:
num_samples = 1000

In [13]:

# Generate random data
data = {
    'user_id': np.random.choice(user_ids, num_samples, replace=True),
    'product_id': np.random.choice(product_ids, num_samples, replace=True),
    'event_timestamp': [random_date(datetime(2022, 1, 1), datetime(2024, 12, 31)) for _ in range(num_samples)],
    'Product_rating': [random.randint(0, 5) for _ in range(num_samples)]
}

# Create DataFrame
df = pd.DataFrame(data)
df["event_timestamp"] = pd.to_datetime(df["event_timestamp"], dayfirst=True)
# Save DataFrame to CSV
df.to_csv(user_observation_source_path, index=False)


In [14]:
# Generate random data
purchase_time = [random_date(datetime(2022, 1, 1), datetime(2024, 12, 31)) for _ in range(num_samples)]
data = {
    'user_id': np.random.choice(user_ids, num_samples, replace=True),
    'purchase_time': purchase_time,
    'purchase_amount': np.random.uniform(1, 150, size=num_samples).round(2),
    'purchase_date': purchase_time
}

# Create DataFrame
df = pd.DataFrame(data)
df["purchase_time"] = pd.to_datetime(df["purchase_time"], dayfirst=True)
df["purchase_date"] = pd.to_datetime(df["purchase_date"], dayfirst=True)
# Save DataFrame to CSV
df.to_csv(user_purchase_history_source_path, index=False)


## 4. Define Sharable Features using Feathr API

### Understand raw datasets
We have three datasets to work with:
* Observation dataset (a.k.a. labeled dataset)
* User profile
* User purchase history
* Product details

In [15]:
# User profile dataset
# Used to generate user features
pd.read_csv(user_profile_file_path).head()

,user_id,age,gift_card_balance,tax_rate,number_of_credit_cards
0,56,58,1812,2,2
1,101,59,1109,4,1
2,487,22,1760,5,1
3,60,44,701,4,2
4,227,18,866,4,2


In [16]:
# User purchase history dataset.
# Used to generate user features. This is activity type data, so we need to use aggregation to generate features.
pd.read_csv(user_purchase_history_file_path).head()

,user_id,purchase_time,purchase_amount,purchase_date
0,299,2022-07-06,139.75,2022-07-06
1,245,2023-11-09,143.66,2023-11-09
2,312,2024-06-19,30.94,2024-06-19
3,235,2023-01-31,33.96,2023-01-31
4,221,2024-05-18,31.51,2024-05-18


In [17]:
# Product detail dataset.
# Used to generate product features.
pd.read_csv(product_detail_file_path).head()

,product_id,price,quantity
0,12,159.582276,48
1,67,100.176701,64
2,88,61.170795,25
3,90,111.296690,82
4,79,66.178578,30


### What's a feature in Feathr
A feature is an individual measurable property or characteristic of a phenomenon which is sometimes time-sensitive.

In Feathr, a feature is defined by the following characteristics:
* The typed key (a.k.a. entity id): identifies the subject of feature, e.g. a user id of 123, a product id of SKU234456.
* The feature name: the unique identifier of the feature, e.g. user_age, total_spending_in_30_days.
* The feature value: the actual value of that aspect at a particular time, e.g. the feature value of the person's age is 30 at year 2022.
* The timestamp: this indicates when the event happened. For example, the user purchased certain product on a certain timestamp. This is usually used for point-in-time join.

You can feel that this is defined from a feature consumer (a person who wants to use a feature) perspective. It only tells us what a feature is like. In later sections, you can see how a feature consumer can access the features in a very simple way.

To define how to produce the feature, we need to specify:
* Feature source: what source data that this feature is based on
* Transformation: what transformation is used to transform the source data into feature. Transformation can be optional when you just want to take a column out from the source data.

(For more details on feature definition, please refer to the [Feathr Feature Definition Guide](https://feathr-ai.github.io/feathr/concepts/feature-definition.html).)

Note: in some cases, such as features defined on top of request data, may have no entity key or timestamp.
It is merely a function/transformation executing against request data at runtime.
For example, the day of week of the request, which is calculated by converting the request UNIX timestamp.
(We won't cover this in the tutorial.)

### Define Sources Section with UDFs

A feature is called an anchored feature when the feature is directly extracted from the source data, rather than computed on top of other features. The latter case is called derived feature.

A [feature source](https://feathr.readthedocs.io/en/latest/#feathr.Source) is needed for anchored features that describes the raw data in which the feature values are computed from. See the python documentation to get the details on each input column.

See [the python API documentation](https://feathr.readthedocs.io/en/latest/#feathr.HdfsSource) to get the details of each input fields.

In [18]:
def feathr_udf_preprocessing(df: DataFrame) -> DataFrame:
    from pyspark.sql.functions import col

    return df.withColumn("tax_rate_decimal", col("tax_rate") / 100)

batch_source = HdfsSource(
    name="userProfileData",
    path=user_profile_source_path,
    preprocessing=feathr_udf_preprocessing,
)

In [19]:
# Let's define some features for users so our recommendation can be customized for users.
user_id = TypedKey(
    key_column="user_id",
    key_column_type=ValueType.INT32,
    description="user id",
    full_name="product_recommendation.user_id",
)

feature_user_age = Feature(
    name="feature_user_age",
    key=user_id,
    feature_type=INT32,
    transform="age",
)
feature_user_tax_rate = Feature(
    name="feature_user_tax_rate",
    key=user_id,
    feature_type=FLOAT,
    transform="tax_rate_decimal",
)
feature_user_gift_card_balance = Feature(
    name="feature_user_gift_card_balance",
    key=user_id,
    feature_type=FLOAT,
    transform="gift_card_balance",
)
feature_user_has_valid_credit_card = Feature(
    name="feature_user_has_valid_credit_card",
    key=user_id,
    feature_type=BOOLEAN,
    transform="number_of_credit_cards > 0",
)

features = [
    feature_user_age,
    feature_user_tax_rate,
    feature_user_gift_card_balance,
    feature_user_has_valid_credit_card,
]

user_feature_anchor = FeatureAnchor(
    name="anchored_features", source=batch_source, features=features
)

In [20]:
# Let's define some features for the products so our recommendation can be customized for products.
product_batch_source = HdfsSource(
    name="productProfileData",
    path=product_detail_source_path,
)

product_id = TypedKey(
    key_column="product_id",
    key_column_type=ValueType.INT32,
    description="product id",
    full_name="product_recommendation.product_id",
)

feature_product_quantity = Feature(
    name="feature_product_quantity",
    key=product_id,
    feature_type=FLOAT,
    transform="quantity",
)
feature_product_price = Feature(
    name="feature_product_price", key=product_id, feature_type=FLOAT, transform="price"
)

product_features = [feature_product_quantity, feature_product_price]

product_feature_anchor = FeatureAnchor(
    name="product_anchored_features",
    source=product_batch_source,
    features=product_features,
)

### Define window aggregation features

[Window aggregation](https://en.wikipedia.org/wiki/Window_function_%28SQL%29) helps us to create more powerful features by compressing large amount of information. For example, we can compute *average purchase amount over the last 90 days* from the purchase history to capture user's recent consumption trend.

To create window aggregation features, we define `WindowAggTransformation` with following arguments:
1. `agg_expr`: the field/column you want to aggregate. It can be an ANSI SQL expression, e.g. `cast_float(purchase_amount)` to cast `str` type values to `float`.
2. `agg_func`: the aggregation function, e.g. `AVG`. See below table for the full list of supported functions.
3. `window`: the aggregation window size, e.g. `90d` to aggregate over the 90 days.

| Aggregation Type | Input Type | Description |
| --- | --- | --- |
| `SUM`, `COUNT`, `MAX`, `MIN`, `AVG` | Numeric | Applies the the numerical operation on the numeric inputs. |
| `MAX_POOLING`, `MIN_POOLING`, `AVG_POOLING`	| Numeric Vector | Applies the max/min/avg operation on a per entry basis for a given a collection of numbers. |
| `LATEST` | Any | Returns the latest not-null values from within the defined time window. |

After you have defined features and sources, bring them together to build an anchor:

> Note that if the features comes directly from the observation data, the `source` argument should be `INPUT_CONTEXT` to indicate the source of the anchor is the observation data.

In [21]:
purchase_history_data = HdfsSource(
    name="purchase_history_data",
    path=user_purchase_history_source_path,
    event_timestamp_column="purchase_date",
    timestamp_format="yyyy-MM-dd",
)

agg_features = [
    Feature(
        name="feature_user_avg_purchase_for_90days",
        key=user_id,
        feature_type=FLOAT,
        transform=WindowAggTransformation(
            agg_expr="cast_float(purchase_amount)", agg_func="AVG", window="90d"
        ),
    )
]

user_agg_feature_anchor = FeatureAnchor(
    name="aggregationFeatures", source=purchase_history_data, features=agg_features
)

### Derived Features Section
Derived features are the features that are computed from other features. They could be computed from anchored features or other derived features.

In [22]:
derived_features = [
    DerivedFeature(
        name="feature_user_purchasing_power",
        key=user_id,
        feature_type=FLOAT,
        input_features=[feature_user_gift_card_balance, feature_user_has_valid_credit_card],
        transform="feature_user_gift_card_balance + if(boolean(feature_user_has_valid_credit_card), 100, 0)",
    )
]

### Build features

Lastly, we need to build those features so that it can be consumed later. Note that we have to build both the "anchor" and the "derived" features which is not anchored to a source.

In [23]:
client.build_features(
    anchor_list=[user_agg_feature_anchor, user_feature_anchor, product_feature_anchor],
    derived_feature_list=derived_features,
)

## Feature Registration
Lastly, we can also register the features and share them across teams:

In [24]:
try:
    client.register_features()
except Exception as e:
    print(e)
print(client.list_registered_features(project_name=PROJECT_NAME))

[{'name': 'feature_user_age', 'id': '56d0a8e0-ee1e-4d4a-9dd4-7a86b759ddb7', 'qualifiedName': 'product_recommendation__anchored_features__feature_user_age'}, {'name': 'feature_product_quantity', 'id': '46783279-8804-4a49-aec1-144e2480682c', 'qualifiedName': 'product_recommendation__product_anchored_features__feature_product_quantity'}, {'name': 'feature_user_tax_rate', 'id': '9e285cd3-1349-4b32-925a-687193a6c383', 'qualifiedName': 'product_recommendation__anchored_features__feature_user_tax_rate'}, {'name': 'feature_product_price', 'id': '5208e000-14b5-44df-9552-4e6bcddc53c2', 'qualifiedName': 'product_recommendation__product_anchored_features__feature_product_price'}, {'name': 'feature_user_gift_card_balance', 'id': 'e802dce6-9bd0-494d-afd1-c3fb0673443b', 'qualifiedName': 'product_recommendation__anchored_features__feature_user_gift_card_balance'}, {'name': 'feature_user_purchasing_power', 'id': 'b729c70e-c594-4b56-a5b4-9ed9a56b1798', 'qualifiedName': 'product_recommendation__feature_u

In [25]:
client.list_registered_features(project_name=PROJECT_NAME)

[{'name': 'feature_user_age',
  'id': '56d0a8e0-ee1e-4d4a-9dd4-7a86b759ddb7',
  'qualifiedName': 'product_recommendation__anchored_features__feature_user_age'},
 {'name': 'feature_product_quantity',
  'id': '46783279-8804-4a49-aec1-144e2480682c',
  'qualifiedName': 'product_recommendation__product_anchored_features__feature_product_quantity'},
 {'name': 'feature_user_tax_rate',
  'id': '9e285cd3-1349-4b32-925a-687193a6c383',
  'qualifiedName': 'product_recommendation__anchored_features__feature_user_tax_rate'},
 {'name': 'feature_product_price',
  'id': '5208e000-14b5-44df-9552-4e6bcddc53c2',
  'qualifiedName': 'product_recommendation__product_anchored_features__feature_product_price'},
 {'name': 'feature_user_gift_card_balance',
  'id': 'e802dce6-9bd0-494d-afd1-c3fb0673443b',
  'qualifiedName': 'product_recommendation__anchored_features__feature_user_gift_card_balance'},
 {'name': 'feature_user_purchasing_power',
  'id': 'b729c70e-c594-4b56-a5b4-9ed9a56b1798',
  'qualifiedName': 'prod

In [26]:
feature_dict = client.get_features_from_registry(project_name=PROJECT_NAME, return_keys=True, verbose=True)

2024-11-11 17:30:14.293 | INFO     | feathr.client:get_features_from_registry:1282 - Get anchor features from registry: 
2024-11-11 17:30:14.294 | INFO     | feathr.client:get_features_from_registry:1288 - {
  "name": "feature_user_avg_purchase_for_90days",
  "featureType": {
    "type": "TENSOR",
    "tensorCategory": "DENSE",
    "dimensionType": [],
    "valType": "FLOAT"
  },
  "key": [
    {
      "keyColumn": "user_id",
      "keyColumnType": "INT",
      "fullName": "product_recommendation.user_id",
      "description": "product_recommendation.user_id",
      "keyColumnAlias": "user_id"
    }
  ],
  "transformation": {
    "defExpr": "cast_float(purchase_amount)",
    "aggFunc": "AVG",
    "window": "90d"
  }
}
2024-11-11 17:30:14.295 | INFO     | feathr.client:get_features_from_registry:1288 - {
  "name": "feature_user_tax_rate",
  "featureType": {
    "type": "TENSOR",
    "tensorCategory": "DENSE",
    "dimensionType": [],
    "valType": "FLOAT"
  },
  "key": [
    {
      "k

## we can list all features

In [27]:
[feat.name for feat in list(feature_dict[0].values())]

['feature_user_avg_purchase_for_90days',
 'feature_user_tax_rate',
 'feature_user_age',
 'feature_user_gift_card_balance',
 'feature_user_has_valid_credit_card',
 'feature_product_price',
 'feature_product_quantity',
 'feature_user_purchasing_power']

## we can list all type_key

In [28]:
[type_key.key_column for type_keys in list(feature_dict[1].values()) for type_key in type_keys]

['user_id',
 'user_id',
 'user_id',
 'user_id',
 'user_id',
 'product_id',
 'product_id',
 'user_id']

We can then get the features from the online store (Redis):

You should be able to see the Feathr UI by visiting the website below:

http://localhost:8081/

In [29]:
from IPython.display import IFrame
IFrame("http://localhost:8081/projects", 900,500)